# Project 2: Boulder Open Data
This notebook explores possible data sources for Project 2.

**Informational URL:** https://bouldercolorado.gov/open-data/search/?p=1&q=osmp

**Marty help:** https://github.com/mchandler13/BoulderView/blob/master/src/pull.py

In [2]:
import requests
import pandas as pd

# Visitor Master Plan

**Informational URL**: 
https://bouldercolorado.gov/open-data/city-of-boulder-osmp-visitor-master-plan/

In [3]:
url_master_plan = "https://www-static.bouldercolorado.gov/docs/opendata/OSMPManagementAreas.GeoJSON?_ga=2.177582950.2110561214.1553967684-646607836.1553818032"

# response.encoding = 'utf-8'
#data = data.content.decode('utf-8') 

data = requests.get(url_master_plan)
data = data.json().get("features")
df = pd.DataFrame.from_records(data)

# Break out data nested in columns into individual columns.
# Extract data from dictionaries within each dataframe column.
# Add those values as new columns.
df2 = df["geometry"].apply(pd.Series)
df3 = df["properties"].apply(pd.Series)

# We do not concatenate the original data back to the dataframe.
# Thus, df is missing from the next line.
df = pd.concat([df2, df3], axis=1)

# Add new columns to break numeric keys into quartiles.
# This makes it easier to summarize counts in the next cells.
df['Acres_Qcut'] = pd.qcut(df['Acres'], 4)
df['Hectares_Qcut'] = pd.qcut(df['Hectares'], 4)

df.head()



,coordinates,type,Acres,Active,Hectares,MGMT_AREA,OtherInfo,Shape_area,Shape_len,ZONENAME,Acres_Qcut,Hectares_Qcut
0,"[[[-105.262030799, 40.0727756001], [-105.26201...",Polygon,611.773295,Yes,247.574967,Passive Recreation Area,None,2.664874e+07,63743.031494,Boulder Valley Ranch,"(146.676, 3527.719]","(75.07, 1427.829]"
1,"[[[-105.218311979, 40.0271273377], [-105.21829...",Polygon,47.545223,Yes,19.240873,Natural Area,None,2.071062e+06,11975.552231,Creek Confluence,"(42.312, 146.676]","(2.271, 19.883]"
2,"[[[-105.19535804, 39.9968808738], [-105.195564...",Polygon,68.539902,Yes,27.737115,Passive Recreation Area,None,2.985586e+06,10308.327996,Dry Creek,"(42.312, 146.676]","(19.883, 75.07]"
3,"[[[-105.305335014, 40.0389272482], [-105.30533...",Polygon,352.149495,Yes,142.509183,Natural Area,None,1.533955e+07,27987.459519,West Sanitas,"(146.676, 3527.719]","(75.07, 1427.829]"
4,"[[[-105.243405171, 40.0152373259], [-105.24340...",Polygon,3.396253,Yes,1.374415,Natural Area,None,1.479402e+05,2589.740297,Outlots,"(0.037, 5.612]","(-0.001, 2.271]"


In [5]:
# Explore the data for counts, data types, and nullables.
df.shape

(186, 12)

In [140]:
# And get a sense of counts to see what kind of charts we might be
# able to make.
for c in df.columns[[1,3,5,6,9,10]]:
    print "---- %s ---" % c
    print df[c].value_counts()

---- type ---
Polygon    186
Name: type, dtype: int64
---- Active ---
Yes    148
No      35
Name: Active, dtype: int64
---- MGMT_AREA ---
Natural Area                               87
Habitat Conservation Area                  31
Passive Recreation Area                    29
Agricultural Area                          15
Natural Area - Conditional                  9
Habitat Conservation Area - Conditional     7
Passive Recreation Area - Conditional       5
Agricultural Area - Conditional             2
TBD                                         1
Name: MGMT_AREA, dtype: int64
---- OtherInfo ---
No Public Access    11
Name: OtherInfo, dtype: int64
---- ZONENAME ---
Northern Tier                               19
Creek Confluence                            14
Outlots                                     13
Southern Grasslands                         11
Diagonal                                    10
South Boulder Creek                          9
East Boulder Valley                          9

# OSMP Climbing Formations

**Informational URL**:  https://bouldercolorado.gov/open-data/osmp-climbing-formations/

In [6]:
url_climbing_formations = "https://www-static.bouldercolorado.gov/docs/opendata/OSMPClimbingFormations.GeoJSON?_ga=2.118797002.2110561214.1553967684-646607836.1553818032"

data = requests.get(url_climbing_formations)
data = data.json().get('features')
df = pd.DataFrame.from_records(data)

# Break out data nested in columns into individual columns.
# Extract data from dictionaries within each dataframe column.
# Add those values as new columns.
df2 = df["geometry"].apply(pd.Series)
df3 = df["properties"].apply(pd.Series)

# We do not concatenate the original data back to the dataframe.
# Thus, df is missing from the next line.
df = pd.concat([df2, df3], axis=1)

# Add new columns to break numeric keys into quartiles.
# This makes it easier to summarize counts in the next cells.
df['UseRating_Qcut'] = pd.qcut(df['UseRating'], 4)
df['ROUTES_Qcut'] = pd.qcut(df['ROUTES'], 4, duplicates="drop")

df.head()


,coordinates,type,AKA,AreaAccess,ClosureAct,FEATURE,FormationT,HCA,ID,Label,OWNER,PERMITREQ,ROUTES,SeasonalCl,UseRating,UseRating_Qcut,ROUTES_Qcut
0,"[-105.294224206, 40.0050200439]",Point,First Areas,Flagstaff,N,Pumpkin Rock,Boulder,No,1,11,OSMP,No,4,N,2.50,"(2.0, 3.0]","(-0.001, 34.0]"
1,"[-105.28786089, 39.9752763893]",Point,None,NCAR,N,Veranda,Wall,No,2,186,OSMP,No,2,N,0.00,"(-0.001, 1.0]","(-0.001, 34.0]"
2,"[-105.29359767, 39.9954108421]",Point,None,Gregory Canyon,N,Third Pinnacle,Wall,No,3,12,OSMP,No,7,N,1.25,"(1.0, 1.8]","(-0.001, 34.0]"
3,"[-105.294391407, 39.9863578462]",Point,None,Chautauqua,N,The Fin,Wall,No,4,33,OSMP,No,1,Y,1.17,"(1.0, 1.8]","(-0.001, 34.0]"
4,"[-105.292810793, 39.9959517203]",Point,None,Gregory Canyon,N,First Pinnacle,Wall,No,6,13,OSMP,No,23,N,2.20,"(2.0, 3.0]","(-0.001, 34.0]"


In [7]:
df.shape

(466, 17)

In [165]:
# And get a sense of counts to see what kind of charts we might be
# able to make.

for c in df.columns[[1,2,3,4,5,6,10,11,12,13,15]]:
    print "---- %s ---" % c
    print df[c].value_counts()

---- type ---
Point    466
Name: type, dtype: int64
---- AKA ---
Shadow Canyon Boulders    6
Satellite Boulders        5
North Terrain Boulders    4
Skunk Canyon East         4
Upper Blues Boulders      4
Monkey Traverse Area      3
Terrain Boulders          3
Skunk Canyon West         3
Physical Boulders         2
Metaphysical Boulders     2
Eldo East Draw            2
First Areas               2
Cloudshadow Area          2
Past Monkey Traverse      2
North Wing                1
Physical Boulder          1
Great Ridge               1
South Wing                1
Upper Terrain Boulders    1
Name: AKA, dtype: int64
---- AreaAccess ---
Cragmoor Rd            81
Sacred Cliffs          78
NCAR                   68
Bear Mountain Drive    65
Chautauqua             64
South Mesa Trail       42
Eldo                   23
Flagstaff              15
Eldorado Mountain      12
Gregory Canyon          8
Mt Sanitas              8
Boulder Canyon          2
Name: AreaAccess, dtype: int64
---- ClosureAct 

# OSMP Lands

**Informational URL:** https://bouldercolorado.gov/open-data/city-of-boulder-osmp-lands/

In [8]:
url_lands = "https://www-static.bouldercolorado.gov/docs/opendata/OSMPLands.GeoJSON?_ga=2.72783824.2110561214.1553967684-646607836.1553818032"

data = requests.get(url_lands)
data = data.json().get('features')
df = pd.DataFrame.from_records(data)

# Break out data nested in columns into individual columns.
# Extract data from dictionaries within each dataframe column.
# Add those values as new columns.
df2 = df["geometry"].apply(pd.Series)
df3 = df["properties"].apply(pd.Series)

# We do not concatenate the original data back to the dataframe.
# Thus, df is missing from the next line.
df = pd.concat([df2, df3], axis=1)

# Add new columns to break numeric keys into quartiles.
# This makes it easier to summarize counts in the next cells.
df['Acres_Qcut'] = pd.qcut(df['Acres'], 4)
df['Hectares_Qcut'] = pd.qcut(df['Hectares'], 4)

df.head()


,coordinates,type,AcqID,Acres,BldEnv,Hectares,JOINTPURCH,Manager,NAME,PUBLICACCE,...,Shape_area,Shape_len,T,TSA,TYPE,WhyNoAcces,YEARCLOSE,lbl_Acres,Acres_Qcut,Hectares_Qcut
0,"[[[-105.251681628, 40.1706585478], [-105.25974...",Polygon,933,74.395579,NO,30.106822,n,OSMP,WALDORF,n,...,3.240658e+06,7908.705573,2N,NTSA,Fee Property,New Property,2002,74,"(63.103, 1751.151]","(25.537, 708.666]"
1,"[[[-105.192477352, 40.1705964942], [-105.19248...",Polygon,886,71.330724,NO,28.866520,n,OSMP,OASIS DIRECT SEVEN,n,...,3.107154e+06,7998.047884,2N,NTSA,Fee Property,New Property,2001,71,"(63.103, 1751.151]","(25.537, 708.666]"
2,"[[[-105.23036232, 40.1705114451], [-105.230371...",Polygon,867,83.489831,NO,33.787136,y,Private,LOPEZ Ce,n,...,3.636802e+06,13253.526495,2N,NTSA,Conservation Easement,Conservation Easement,2002,83,"(63.103, 1751.151]","(25.537, 708.666]"
3,"[[[-105.183179544, 40.1492129459], [-105.18316...",Polygon,918,35.619969,NO,14.414890,y,BCPOS,SUITTS,n,...,1.551600e+06,5878.349554,2N,NTSA,Fee Property,BOCO Management,2001,36,"(19.422, 63.103]","(7.86, 25.537]"
4,"[[[-105.183179544, 40.1492129459], [-105.18324...",Polygon,918,35.895468,NO,14.526380,y,BCPOS,SUITTS,n,...,1.563600e+06,7047.015249,2N,NTSA,Fee Property,BOCO Management,2001,36,"(19.422, 63.103]","(7.86, 25.537]"


In [9]:
df.shape

(733, 24)

In [181]:
df.shape

---- BldEnv ---
NO     698
YES     35
Name: BldEnv, dtype: int64
---- JOINTPURCH ---
n    641
y     92
Name: JOINTPURCH, dtype: int64
---- Manager ---
OSMP       489
Private    132
BCPOS       70
CE          42
Name: Manager, dtype: int64
---- NAME ---
SUITTS                                             6
VARRA, EAST                                        6
SUPERIOR ASSOCIATES                                6
SAWHILL PONDS                                      6
SCHNEIDER-IRLLC                                    5
DAMYANOVICH                                        5
BUCKINGHAM PARK                                    5
VARRA, EAST - CE Southeast                         4
CARIBOU RANCH - CE - LAKEWOOD PARCEL               4
VALMONT BUTTE                                      3
ANDERSON                                           3
OAKLEY BE                                          3
JODER II                                           3
DAGLE                                              3
GREEN

# Prairie Dog Colonies

**Informational URL:* https://bouldercolorado.gov/open-data/city-of-boulder-osmp-prairie-dog-colonies/clim

In [10]:
url_prarie_dogs = "https://www-static.bouldercolorado.gov/docs/opendata/OSMPPrairieDogColonies.GeoJSON"
data = requests.get(url_prarie_dogs)
data = data.json().get('features')
df = pd.DataFrame.from_records(data)

# Break out data nested in columns into individual columns.
# Extract data from dictionaries within each dataframe column.
# Add those values as new columns.
df2 = df["geometry"].apply(pd.Series)
df3 = df["properties"].apply(pd.Series)

# We do not concatenate the original data back to the dataframe.
# Thus, df is missing from the next line.
df = pd.concat([df2, df3], axis=1)

# Add new columns to break numeric keys into quartiles.
# This makes it easier to summarize counts in the next cells.
df['Acres_Qcut'] = pd.qcut(df['Acres'], 4)
df['Hectares_Qcut'] = pd.qcut(df['Hectares'], 4)

df.head()


,coordinates,type,Acres,Activity,Comment,Hectares,Manager,Name,PD_ID,Perimeter,Plague,Shape_area,Shape_len,Year,YearAcquir,sdeWildlif,Acres_Qcut,Hectares_Qcut
0,"[[[[-105.25510497573354, 40.1107786117921], [-...",MultiPolygon,30.013797,2.0,None,12.146153,OSMP,Brewbaker,126.0,17407.132776,None,1.307396e+06,17407.132776,2009.0,1996,1.307396e+06,"(11.674, 32.957]","(4.724, 13.337]"
1,"[[[-105.212594791, 40.0936916196], [-105.21255...",Polygon,5.450247,2.0,None,2.205637,OSMP,Cowles,26.0,2875.475502,None,2.374118e+05,2875.475502,2009.0,1991,2.374118e+05,"(3.029, 11.674]","(1.226, 4.724]"
2,"[[[[-105.22302997985106, 39.97012490453621], [...",MultiPolygon,6.346785,2.0,None,2.568453,OSMP,Sams Lane/Van Vleet - west,136.0,4246.332172,None,2.764648e+05,4246.332172,2009.0,1978,2.764648e+05,"(3.029, 11.674]","(1.226, 4.724]"
3,"[[[[-105.18539718806075, 39.9896294967146], [-...",MultiPolygon,47.677422,2.0,None,19.294368,OSMP,Marshall,68.0,10818.604923,None,2.076820e+06,10818.604923,2009.0,1997,2.076820e+06,"(32.957, 565.835]","(13.337, 228.985]"
4,"[[[-105.29348293, 40.0571514749], [-105.293468...",Polygon,16.960038,2.0,None,6.863484,OSMP,ERNI- Moore,38.0,5580.871249,None,7.387763e+05,5580.871249,2009.0,1967,7.387763e+05,"(11.674, 32.957]","(4.724, 13.337]"


In [11]:
df.shape

(1939, 18)

In [15]:
plotData=[]
for name in df2["Name"].unique():
    temp = df2[df2["Name"]==name].copy()
    x = [int(str(i)) for i in list(temp["YearAcquir"])]
    y = list(temp["Acres"])
    to_append={}
    to_append["Name"]=str(name)
    to_append["x"]=x
    to_append["y"]=y
    plotData.append(to_append)
for i in plotData:
    print(i)
    print("")

{'y': [30.0137966, 35.42867811, 42.65149751, 69.74717427, 74.15467534, 87.7371595, 69.02302624, 84.52624334, 43.07920987, 59.80382698, 64.44205452, 11.87272938, 12.92313728, 25.74841479, 19.334556, 34.15708017, 13.39413175, 17.60233277, 10.63083695, 84.93412349, 99.93546715], 'x': [1996, 1996, 1996, 1996, 2005, 2005, 2005, 2007, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 1996, 2007, 2007], 'Name': 'Brewbaker'}

{'y': [5.45024723, 12.81624002, 21.53223103, 28.48965045, 22.92630884, 31.74179776, 33.2207483, 32.29519762, 32.78354479, 31.50111923, 30.12171932, 9.38925526, 2.13110822, 4.47357708, 36.97644063, 31.63620468, 32.09518659, 22.40024496, 28.89495328, 17.49939526, 9.81854807, 32.35508001, 31.96240798], 'x': [1991, 1991, 1991, 1991, 1991, 1991, 1991, 1992, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1992, 1992], 'Name': 'Cowles'}

{'y': [6.34678488, 7.78729026, 12.33744969, 17.53544314, 25.76263072, 21.44493803, 14.1374358, 15.83962

In [34]:
# df.Name.unique().tolist()
for names in "Brewbaker":
    df = df.loc[lambda df: df['Name'] == 'names']
    yearAcquired = df['YearAcquir'].tolist()
    ttlAcres = df['Acres'].tolist()
    print(df)
    break

Empty DataFrame
Columns: [YearAcquir, Name, Acres]
Index: []


In [187]:
# And get a sense of counts to see what kind of charts we might be
# able to make.

for c in df.columns[[1,3,6,10,13,14,16,17]]:
    print "---- %s ---" % c
    print df[c].value_counts()

---- type ---
MultiPolygon    1086
Polygon          853
Name: type, dtype: int64
---- Activity ---
2.0    1939
Name: Activity, dtype: int64
---- Manager ---
OSMP    1939
Name: Manager, dtype: int64
---- Plague ---
2006         39
2005          8
2007          7
2008          5
NO            3
2009          2
likely        1
1999          1
suspected     1
Name: Plague, dtype: int64
---- Year ---
2016.0    112
2015.0    110
2018.0    109
2012.0    108
2017.0    108
2011.0    101
2013.0    101
2014.0    100
2008.0    100
2009.0     95
2010.0     95
2005.0     89
2007.0     88
2002.0     86
2003.0     83
2004.0     81
2001.0     78
2006.0     74
2000.0     68
1999.0     51
1998.0     47
1997.0     38
1996.0     17
Name: Year, dtype: int64
---- YearAcquir ---
1992    135
1984    135
1973    135
1990    114
1974     92
1999     85
2000     83
1993     82
1985     78
1978     78
1977     75
1995     75
1991     72
2001     69
1998     68
1975     62
1997     62
2006     52
1988     48
1968  

# OSMP Trailheads

**Informational URL:* https://bouldercolorado.gov/open-data/city-of-boulder-osmp-trailheads/

In [194]:
url_trailheads = "https://www-static.bouldercolorado.gov/docs/opendata/OSMPTrailheads.GeoJSON?_ga=2.239917760.2110561214.1553967684-646607836.1553818032"

data = requests.get(url_trailheads)
data = data.json().get('features')
df = pd.DataFrame.from_records(data)

# Break out data nested in columns into individual columns.
# Extract data from dictionaries within each dataframe column.
# Add those values as new columns.
df2 = df["geometry"].apply(pd.Series)
df3 = df["properties"].apply(pd.Series)

# We do not concatenate the original data back to the dataframe.
# Thus, df is missing from the next line.
df = pd.concat([df2, df3], axis=1)

# Add new columns to break numeric keys into quartiles.
# This makes it easier to summarize counts in the next cells.
#df['Acres_Qcut'] = pd.qcut(df['Acres'], 4)
#df['Hectares_Qcut'] = pd.qcut(df['Hectares'], 4)

df.head()




,coordinates,type,ADAcamping,ADAfacName,ADAfacilit,ADAfishing,ADAparking,ADApicnic,ADAsurface,ADAtoilet,...,FISHING,Fee,Grills,HorseTrail,PICNIC,ParkSpaces,RESTROOMS,RecycleBin,THLeash,TrashCans
0,"[-105.306512592, 40.0050207871, 1.4107668579e-05]",Point,No,Wood Shelter,Yes,No,Yes,Yes,Asphalt,Yes,...,No,Yes,Yes,Not Recommended,Yes,12,Yes,Yes,Yes,4.0
1,"[-105.303488566, 40.004187738, 2088.98409336]",Point,No,Nature Center,Yes,No,Yes,Yes,Asphalt,Yes,...,No,Yes,Yes,Not Recommended,Yes,56,Yes,Yes,Yes,6.0
2,"[-105.131779303, 40.0620347568, 1579.71470042]",Point,No,None,No,No,No,No,No,No,...,No,No,No,Not Recommended,No,6,No,No,Yes,1.0
3,"[-105.306433742, 40.111297871, 1813.40045099]",Point,No,None,No,No,Yes,Yes,Crusher Fines,Yes,...,Yes,No,Yes,Possible,Yes,35,Yes,Yes,Yes,2.0
4,"[-105.295324303, 40.0140342671, 1651.04904637]",Point,No,None,No,No,Yes,Yes,Cement/Crusher,No,...,Yes,No,No,Not Recommended,Yes,27,No,Yes,Yes,2.0


In [204]:
# Explore the data for counts, data types, and nullables.
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 33 columns):
coordinates    37 non-null object
type           37 non-null object
ADAcamping     37 non-null object
ADAfacName     12 non-null object
ADAfacilit     37 non-null object
ADAfishing     37 non-null object
ADAparking     37 non-null object
ADApicnic      37 non-null object
ADAsurface     37 non-null object
ADAtoilet      37 non-null object
ADAtrail       37 non-null object
AKA            4 non-null object
AccessID       37 non-null object
AccessName     37 non-null object
AccessType     37 non-null object
Address        37 non-null object
BikeRack       37 non-null object
BikeTrail      37 non-null object
Class          37 non-null object
DateFrom       37 non-null object
DateTo         37 non-null object
DogCompost     37 non-null object
DogTube        37 non-null float64
FISHING        37 non-null object
Fee            37 non-null object
Grills         37 non-null object
HorseTrail   

In [207]:
# And get a sense of counts to see what kind of charts we might be
# able to make.

for c in df.columns[1:]:
    print "---- %s ---" % c
    print df[c].value_counts()

---- type ---
Point    37
Name: type, dtype: int64
---- ADAcamping ---
No     36
Yes     1
Name: ADAcamping, dtype: int64
---- ADAfacName ---
Interpretive Signs         2
Overlook                   2
Ranger Cottage             1
Dock/Shelter/Bird Blind    1
Foothills Nature Center    1
Fishing Dock               1
Cherryvale Offices         1
Nature Center              1
Wood Shelter               1
Overlook/Fee Station       1
Name: ADAfacName, dtype: int64
---- ADAfacilit ---
No     23
Yes    14
Name: ADAfacilit, dtype: int64
---- ADAfishing ---
No     30
Yes     7
Name: ADAfishing, dtype: int64
---- ADAparking ---
Yes        27
No          7
Offsite     2
Yes x 2     1
Name: ADAparking, dtype: int64
---- ADApicnic ---
No     20
Yes    17
Name: ADApicnic, dtype: int64
---- ADAsurface ---
Crusher Fines        15
No                    7
Roadbase/Crusher      7
Asphalt               2
Roadebase/Crusher     2
Cement/Crusher        1
Cement                1
Roadebase             1
Roadbas

# OSMP Trails

**Informational URL:* https://bouldercolorado.gov/open-data/city-of-boulder-osmp-trails/

In [212]:
url_trails = "https://www-static.bouldercolorado.gov/docs/opendata/OSMPTrails.GeoJSON?_ga=2.239917760.2110561214.1553967684-646607836.1553818032"

data = requests.get(url_trails)
data = data.json().get('features')
df = pd.DataFrame.from_records(data)

# Break out data nested in columns into individual columns.
# Extract data from dictionaries within each dataframe column.
# Add those values as new columns.
df2 = df["geometry"].apply(pd.Series)
df3 = df["properties"].apply(pd.Series)

# We do not concatenate the original data back to the dataframe.
# Thus, df is missing from the next line.
df = pd.concat([df2, df3], axis=1)

# Add new columns to break numeric keys into quartiles.
# This makes it easier to summarize counts in the next cells.
df['Mileage_Qcut'] = pd.qcut(df['Mileage'], 4)

df.head()


,coordinates,type,Bicycles,DateFrom,DateTo,Display,Horses,LastGPSd,MeasuredFe,Mileage,OWNER,RID,RegionalNa,SegmentID,Shape_len,TrailName,TrailType,TrlID,WhyChanged,Mileage_Qcut
0,"[[-105.40584827, 40.0046769209, 1.920575596e-0...",LineString,No,2005/12/31,2099/12/31,Yes,No,2016/08/31,-9999,0.106,OSMP,1000,None,296-357-356,562.147648,Boulder Falls,Hiking Trail,296,Re-GPS'd to get better data,"(0.0368, 0.108]"
1,"[[-105.238779788, 39.9236461053, 1.920575596e-...",LineString,Yes,2016/10/26,2099/12/31,Yes,Yes,2016/08/31,-9999,0.876,OSMP,1135,None,116-424-421,4625.547170,Flatirons Vista North,Multi-Use Trail,116,Re-GPS'd to get better data,"(0.312, 2.779]"
2,"[[-105.235685749, 39.9240658734, 1.920575596e-...",LineString,Yes,2016/10/25,2099/12/31,Yes,Yes,2016/08/31,-9999,0.019,OSMP,1140,None,116-430-428,102.623588,Flatirons Vista North,Multi-Use Trail,116,Re-GPS'd to get better data,"(0.002, 0.0368]"
3,"[[-105.236550299, 39.9255403946, 1.920575596e-...",LineString,Yes,2016/10/25,2099/12/31,Yes,Yes,2016/08/31,-9999,0.178,OSMP,1544,None,116-425-424,939.742819,Flatirons Vista North,Multi-Use Trail,116,Re-GPS'd to get better data,"(0.108, 0.312]"
4,"[[-105.235547364, 39.9243026049, 1.920575596e-...",LineString,Yes,2016/10/25,2099/12/31,Yes,Yes,2016/08/31,-9999,0.004,OSMP,1141,None,117-429-428,22.315176,Prairie Vista,Multi-Use Trail,117,Re-GPS'd to get better data,"(0.002, 0.0368]"


In [213]:
# Explore the data for counts, data types, and nullables.
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572 entries, 0 to 571
Data columns (total 20 columns):
coordinates     572 non-null object
type            572 non-null object
Bicycles        572 non-null object
DateFrom        572 non-null object
DateTo          572 non-null object
Display         572 non-null object
Horses          572 non-null object
LastGPSd        572 non-null object
MeasuredFe      572 non-null int64
Mileage         572 non-null float64
OWNER           572 non-null object
RID             572 non-null int64
RegionalNa      572 non-null object
SegmentID       572 non-null object
Shape_len       572 non-null float64
TrailName       572 non-null object
TrailType       572 non-null object
TrlID           572 non-null int64
WhyChanged      572 non-null object
Mileage_Qcut    572 non-null category
dtypes: category(1), float64(2), int64(3), object(14)
memory usage: 85.6+ KB


In [218]:
# And get a sense of counts to see what kind of charts we might be
# able to make.

for c in df.columns[1:]:
    print "---- %s ---" % c
    print df[c].value_counts()

---- type ---
Point    466
Name: type, dtype: int64
---- AKA ---
Shadow Canyon Boulders    6
Satellite Boulders        5
North Terrain Boulders    4
Skunk Canyon East         4
Upper Blues Boulders      4
Monkey Traverse Area      3
Terrain Boulders          3
Skunk Canyon West         3
Physical Boulders         2
Metaphysical Boulders     2
Eldo East Draw            2
First Areas               2
Cloudshadow Area          2
Past Monkey Traverse      2
North Wing                1
Physical Boulder          1
Great Ridge               1
South Wing                1
Upper Terrain Boulders    1
Name: AKA, dtype: int64
---- AreaAccess ---
Cragmoor Rd            81
Sacred Cliffs          78
NCAR                   68
Bear Mountain Drive    65
Chautauqua             64
South Mesa Trail       42
Eldo                   23
Flagstaff              15
Eldorado Mountain      12
Gregory Canyon          8
Mt Sanitas              8
Boulder Canyon          2
Name: AreaAccess, dtype: int64
---- ClosureAct 